In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]=""

In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
from tqdm import tqdm
import torch
import sys
import time
import datetime
from itertools import product
import os
import numpy as np
from sklearn import tree
from sklearn import linear_model
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, f1_score
import matplotlib.pyplot as plt
from functools import partial
import shelve
import tqdm
import pickle
from collections import Counter

In [3]:
from ogb.graphproppred import Evaluator
from tdc.benchmark_group import admet_group

/home/michlampert/magisterka/venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%autoreload 2
from huggingmolecules import MatConfig, MatFeaturizer, MatModel
from huggingmolecules import GroverConfig, GroverFeaturizer, GroverModel
from huggingmolecules import RMatConfig, RMatFeaturizer, RMatModel
from models import StrippedMatModel, StrippedGroverModel, StrippedRMatModel

In [5]:
%autoreload 2
from utils import *
from datasets import *
import predict_procedure as pp
import embedding_procedure as ep

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [6]:
# from mat import MolecularAttentionTransformer

# model = MolecularAttentionTransformer()
# model.predict(["CN1CC[C@]23C(=O)C[C@@H]4C(=CCO[C@H]5CC(=O)N(c6ccccc62)[C@@H]3[C@@H]54)C1"]) ## tooks 11m 4s (!)

In [7]:
# group = admet_group(path = 'data/')
# benchmark = group.get("CYP2D6_Substrate_CarbonMangels")

# dataset = benchmark["test"]

# print(dataset[dataset.Drug == "CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C"].to_markdown())

# Output:
# |     | smiles                             |   Y |
# |----:|:-----------------------------------|----:|
# |  58 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   1 |
# | 226 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   0 |else 'regression'

In [29]:
import pathlib

calculated = set(map(str, pathlib.Path("embeddings/").glob("*/*.zip")))

pairs = product(get_dataset_names(), get_embedder_names())

with open("embedding_params.txt", "w") as f:
    for dataset_name, model_name in pairs:
        if f"embeddings/{dataset_name}/{model_name}.zip" not in calculated:
            print(dataset_name, model_name, file=f)


In [7]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = list(product(get_regression_names(), ["SELFormer"]))
calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

params = list(product(list(set(pairs)), ["forest"]))


with open("predict_params.txt", "w") as f:
    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" not in done:
            print(dataset_name, model_name, head_model, file=f)
            

In [36]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = list(product(get_dataset_names(), get_embedder_names()))
calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

params = list(product(list(set(pairs) & set(calculated)), ["ridge","forest"]))


with open("predict_params.txt", "w") as f:
    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" in done and dataset_name in get_classification_names():
            print(dataset_name, model_name, head_model, file=f)

In [19]:
dataset_name, model_name = "HIA_Hou", "molbert"

df, _ = get_dataset(dataset_name)

model = get_embedder(model_name)

ep.procedure(dataset_name, df, model_name, model, max_samples=10)

Found local copy...
Loading...
Done!
Found local copy...


Processing HIA_Hou with molbert


INFO: Processing HIA_Hou with molbert
100%|██████████| 10/10 [00:04<00:00,  2.34it/s]

Processing HIA_Hou with molbert took 0.1m



INFO: Processing HIA_Hou with molbert took 0.1m


In [ ]:
for dataset_name in get_dataset_names():
    get_dataset(dataset_name)[0].reset_index()[["index", "smiles"]].rename(columns={"index": "chembl_id", "smiles": "canonical_smiles"}).to_csv(f"smiles_txt/{dataset_name}.txt", sep='\t', index=False)


In [ ]:
for dataset_name in get_dataset_names():
    os.system(f"python SELFormer/generate_selfies.py --smiles_dataset=smiles_txt/{dataset_name}.txt --selfies_dataset=selfies/{dataset_name}.csv")

In [ ]:
with open("selformer_params.txt", "w") as f:
    for dataset_name in get_dataset_names():
        if len(get_dataset(dataset_name)[0]) < 10000:
            f.write(f"{dataset_name}\n")
    

In [8]:
a = pd.read_csv("selformer_embeddings/Half_Life_Obach.csv").rename(columns={"chembl_id": "index"})
a

,index,sequence_embeddings
0,0,"[1.2678825855255127, -0.04146221652626991, -0...."
1,1,"[0.8832750916481018, -0.09156806021928787, -0...."
2,2,"[1.2266165018081665, -0.035363443195819855, -0..."
3,3,"[1.1679654121398926, 0.12847484648227692, -0.4..."
4,4,"[1.3297243118286133, 0.20555377006530762, -0.3..."
...,...,...
662,662,"[1.3313795328140259, -0.08630513399839401, -0...."
663,663,"[1.2012218236923218, 0.008925149217247963, -0...."
664,664,"[1.079952359199524, -0.17504577338695526, -0.5..."
665,665,"[1.0526909828186035, 0.13809537887573242, -0.4..."


In [9]:
b = get_dataset("Half_Life_Obach")[0].reset_index()
b

Found local copy...
Loading...
Done!
Found local copy...


,index,smiles,Y
0,0,CCN1CC(CCN2CCOCC2)C(c2ccccc2)(c2ccccc2)C1=O,4.10
1,1,COC(=O)C1=C(C)NC(C)=C(C(=O)OCCN(C)Cc2ccccc2)C1...,4.10
2,2,CCN(CC)CCNC(=O)c1ccc(NC(C)=O)cc1,6.40
3,3,CN1[C@H]2CC[C@@H]1C[C@H](OC(=O)C(CO)c1ccccc1)C2,4.10
4,4,O=c1[nH]cc(F)c(=O)[nH]1,0.12
...,...,...,...
662,662,CC(=O)Nc1c(I)c(NC(C)=O)c(I)c(C(=O)O)c1I,1.80
663,663,CN(C)CCc1c[nH]c2ccc(C[C@H]3COC(=O)N3)cc12,3.60
664,664,COc1ccc2cc1Oc1cc3c(cc1OC)CC[N+](C)(C)[C@H]3Cc1...,5.10
665,665,COc1cccc(C2(O)CCCCC2CN(C)C)c1,5.80


In [25]:
c = a.join(b, on="index", lsuffix="l", rsuffix="r").drop(["indexl", "indexr"], axis=1).rename(columns={"sequence_embeddings": "embeddings"})
c["embeddings"] = c["embeddings"].map(eval)
len(c.embeddings[100])

768

In [10]:
from deepchem.feat.graph_data import BatchGraphData
from deepchem.feat.molecule_featurizers.conformer_featurizer import RDKitConformerFeaturizer
from deepchem.models.torch_models.gnn3d import InfoMax3DModular
import numpy as np
import deepchem as dc
from deepchem.data.datasets import NumpyDataset

smiles = ["C[C@H](F)Cl", "C[C@@H](F)Cl"]
featurizer = RDKitConformerFeaturizer()
data = featurizer.featurize(smiles)
dataset = NumpyDataset(X=data)
model = InfoMax3DModular(task='regression',
                         hidden_dim=64,
                         target_dim=10,
                         aggregators=['max'],
                         readout_aggregators=['mean'],
                         scalers=['identity'])

INFO: Featurizing datapoint 0


In [20]:
pd.read_pickle("embeddings/Half_Life_Obach/mol2vec.zip").embeddings.dtype

dtype('O')